In [1]:
import datetime

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
# https://data.rivm.nl/covid-19/
data = pd.read_csv('COVID-19_aantallen_gemeente_per_dag.csv', sep=';')
data

,Version,Date_of_report,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Deceased
0,4,2022-03-10 10:00:00,2020-02-28,GM0014,Groningen,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0
1,4,2022-03-10 10:00:00,2020-02-28,GM0034,Almere,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0
2,4,2022-03-10 10:00:00,2020-02-28,GM0037,Stadskanaal,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0
3,4,2022-03-10 10:00:00,2020-02-28,GM0047,Veendam,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0
4,4,2022-03-10 10:00:00,2020-02-28,GM0050,Zeewolde,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
276761,4,2022-03-10 10:00:00,2022-03-10,NaN,NaN,Limburg,VR24,Limburg-Zuid,GGD Zuid-Limburg,Netwerk Acute Zorg Limburg,44,0
276762,4,2022-03-10 10:00:00,2022-03-10,NaN,NaN,Zuid-Holland,VR15,Haaglanden,GGD Haaglanden,Netwerk Acute Zorg West,26,0
276763,4,2022-03-10 10:00:00,2022-03-10,NaN,NaN,Fryslân,VR02,Fryslân,GGD Fryslân,Acute Zorgnetwerk Noord Nederland,0,0
276764,4,2022-03-10 10:00:00,2022-03-10,NaN,NaN,Noord-Holland,VR11,Zaanstreek-Waterland,GGD Zaanstreek/Waterland,NaN,2,0


In [3]:
daily_cases_sr = data.groupby(['Security_region_code', 'Date_of_publication'])['Total_reported'].sum()
daily_cases_sr

Security_region_code  Date_of_publication
VR01                  2020-02-28                0
                      2020-02-29                0
                      2020-03-01                0
                      2020-03-02                0
                      2020-03-03                0
                                             ... 
VR25                  2022-03-06              905
                      2022-03-07              906
                      2022-03-08             1031
                      2022-03-09             1089
                      2022-03-10              892
Name: Total_reported, Length: 18550, dtype: int64

In [4]:
rolling_mean_sr = daily_cases_sr.rolling(7).mean().rename('Rolling_mean')
rolling_mean_sr

Security_region_code  Date_of_publication
VR01                  2020-02-28                    NaN
                      2020-02-29                    NaN
                      2020-03-01                    NaN
                      2020-03-02                    NaN
                      2020-03-03                    NaN
                                                ...    
VR25                  2022-03-06             804.285714
                      2022-03-07             846.428571
                      2022-03-08             887.857143
                      2022-03-09             933.857143
                      2022-03-10             949.714286
Name: Rolling_mean, Length: 18550, dtype: float64

In [8]:
sr_codes = ['VR15', 'VR17']

def gen_sr_plot(sr, avg):
    if avg:
        kwargs = {'fill': 'tozeroy', 'mode': 'none'}
    else:
        kwargs = {}
    return go.Scatter(x=sr.index.droplevel('Security_region_code'), y=sr.array, name=sr.index[0][0],
        **kwargs)

today = datetime.date.fromisoformat(daily_cases_sr.last_valid_index()[1])
yesteryear = datetime.date(today.year - 1, today.month, today.day)

fig = go.Figure()
fig.add_traces(daily_cases_sr.loc[sr_codes].groupby('Security_region_code').apply(gen_sr_plot, avg=True)
    .tolist())
fig.add_traces(rolling_mean_sr.loc[sr_codes].groupby('Security_region_code').apply(gen_sr_plot, avg=False)
    .tolist())
fig.update_layout(xaxis_range=[yesteryear,today], yaxis_range=[0,15000])
fig.show()